<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories that are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-512 Image Processing II**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**: 
    [Pol del Aguila Pla](mailto:pol.delaguilapla@epfl.ch), 
    [Kay Lächler](mailto:kay.lachler@epfl.ch),
    [Alejandro Noguerón Arámburu](mailto:alejandro.nogueronaramburu@epfl.ch),
    [Kamil Seghrouchni](mailto:kamil.seghrouchni@epfl.ch), and
    [Daniel Sage](mailto:daniel.sage@epfl.ch).
    
---
# Lab 5.2: Geometric transformation - Applications
**Released**: Thursday, March 13, 2025

**Submission deadline**: Monday, March 24, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=463)

**Grade weight**: Lab 5 (22 points), 7.5 % of the overall grade

**Related lectures**: Chapter 7

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
import getpass
import imageio.v3 as imageio
alsace = imageio.imread('images/alsace.png') / 255.
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

# Geometric transformation applications  (8 points)

We compare the interpolators studied in Part 1 in more detail, and then use geometric transformations for several applications.

# Comparison of interpolators (3 points)

In Part 1, we visually inspected the differences between the interpolation methods.
Now, we turn to a numerical comparison.
First, we implement a utility function that wraps geometric transformations with a simple interface. 

**For 2 points**, implement `transform` taking the arguments

* `img`: the original image,
* `angle`: the angle of rotation of the coordinate frame in radians,
* `scale`: the scaling factor,
* `center`: the center of rotation,
* `order`: the order of interpolation to use (`0`=nearest neighbor, `1`=linear, and `3`=cubic),

and returning

* `out`: the transformed image.

The function can be easily implemented with building blocks from Part 1; the heavy lifting should be done with scipy's `ndimage.affine_transform`.

In [ ]:
import numpy as np
import scipy.ndimage as nd
import cv2 as cv

def transform(img, angle, scale, center, order):
    out = np.zeros(img.shape)
    
    # YOUR CODE HERE
    
    return out

Run the next cell for a quick sanity check. In it, we also define the function `SNR` that calculates the signal-to-noise ratio between two images, which we will use for the upcoming analysis.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from interactive_kit import imviewer as viewer

test_img = np.zeros((7, 7))
test_img[1:6, 1:6] = 255
test_img[2:5, 2:5] = 0
reference = np.array([[
    [255., 255., 255.,   0.,   0.,   0.,   0.],
    [255.,   0.,   0., 255.,   0.,   0.,   0.],
    [  0.,   0.,   0., 255., 255.,   0.,   0.],
    [255.,   0., 255., 255.,   0.,   0.,   0.],
    [255., 255., 255.,   0.,   0.,   0.,   0.],
    [  0., 255.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.]
], [
    [164., 193., 200.,   0.,   0.,   0.,   0.],
    [146.,   0., 109., 200.,   0.,   0.,   0.],
    [  0.,   0.,   0., 193., 129.,   0.,   0.],
    [182.,   0., 146., 164.,   0.,   0.,   0.],
    [128., 224., 164.,   0.,   0.,   0.,   0.],
    [  0.,  82.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.]
], [
    [171., 259., 215.,  -0.,   0.,   0.,   0.],
    [181., -59., 134., 215.,  -0.,   0.,   0.],
    [-40.,  -4., -59., 259., 115.,   0.,   0.],
    [224., -40., 181., 171.,   0.,   0.,   0.],
    [120., 299., 172.,   0.,   0.,   0.,   0.],
    [  0.,  59.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.]
]])

orders = [0, 1, 3]
names_interp = {0: 'Nearest neighbors', 1: 'Linear', 3: 'Cubic'}
transformed = [np.round(transform(test_img, np.deg2rad(45), 0.9, (0, 3), i)) for i in orders]
names = [f'Transformed, {names_interp[i]}' for i in orders]
plt.close('all')
img_list = []
title_list = []
err_count = 0
for i, o in enumerate(orders):
    if not np.allclose(reference[i], transformed[i], rtol=1e-2):
        print(f'Warning: `transform` with {names_interp[o]} interpolation did not produce the correct result. Check the images below:')
        img_list.append(transformed[i])
        img_list.append(reference[i])
        img_list.append(transformed[i] - reference[i])
        title_list.append(names[i])
        title_list.append(names[i] + ' (correct)')
        title_list.append('Difference')
        err_count += 1
if err_count > 0:
    view = viewer(test_img, title=['Test image'], subplots=(1,1))
    view_err = viewer(img_list, title=title_list, subplots=(err_count, 3))
else:
    view = viewer([test_img] + transformed, title=['Test image'] + names, subplots=(2,2))
    print('Nice! Your transform function passed the sanity check.')
    

To clearly see the differences between interpolation methods, we apply multiple consecutive transformations with the same interpolation.
Here, we stick to rotation and create a function rotating an image $n$ times around its center by an angle $\alpha$.
As an example, when $n=5$ and $\alpha=\pi/10$, we apply $5$ consecutive rotations of $\pi/10$ around the center (as opposed to one rotation by $\pi/2$).

**For 1 point**, implement `rotate_n` taking the arguments

* `img`: the input image,
* `alpha`: the angle of rotation,
* `n`: the number of consecutive rotations,
* `order`: the order of interpolation, as above,

and returning

* `out`: the rotated image.

Make use of `transform`, setting the scale parameter to `1`.

In [ ]:
def rotate_n(img, alpha, n, order):
    out = np.zeros(img.shape)

    # YOUR CODE HERE
    
    return out

Run the next cell to perform a quick sanity check.

In [ ]:
# Define test image
test_img = np.zeros((7,7))
test_img[test_img.shape[0]//2, :] = 255
# Correct results
reference = np.array([[
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.],
    [  0.,   0.,   0., 255.,   0.,   0.,   0.],
    [  0.,   0., 255.,   0.,   0.,   0.,   0.],
    [  0.,   0., 255., 255., 255.,   0.,   0.],
    [  0.,   0., 255., 255., 255.,   0.,   0.],
    [  0.,   0.,   0., 255.,   0.,   0.,   0.],
    [  0.,   0.,   0.,   0.,   0.,   0.,   0.]
], [
    [  0.,   0.,  25.,  56.,   0.,   0.,   0.],
    [  0.,   0.,  26., 161.,  51.,   6.,   0.],
    [  0.,   0.,  57., 121.,  58.,   1.,   0.],
    [  0.,   4.,  63., 255.,  63.,   4.,   0.],
    [  0.,   1.,  58., 121.,  57.,   0.,   0.],
    [  0.,   6.,  51., 161.,  26.,   0.,   0.],
    [  0.,   0.,   0.,  56.,  25.,   0.,   0.]
], [
    [  0.,   0., -13.,  66.,   0.,   0.,   0.],
    [  0.,  -7.,   2., 268.,  31.,  -8.,   0.],
    [  0.,  -8.,  32., 200.,  13., -14.,   2.],
    [  3., -15.,  28., 255.,  28., -15.,   3.],
    [  2., -14.,  13., 200.,  32.,  -8.,   0.],
    [  0.,  -8.,  31., 268.,   2.,  -7.,   0.],
    [  0.,   0.,   0.,  66., -13.,   0.,   0.]
]])
orders = [0, 1, 3]
# Rotate 3 times by 30 degrees
rotated = [np.round(rotate_n(test_img, alpha=np.deg2rad(30), n=3, order=o)) for o in orders]
# Check if correct
names_interp = {0:'Nearest neighbors', 1:'Linear', 3:'Cubic'}
names = [f'Rotated 3x30, {names_interp[i]}' for i in orders]
plt.close('all')
img_list = []
title_list = []
err_count = 0
for i, o in enumerate(orders):
    if not np.allclose(reference[i], rotated[i]):
        print(f'WARNING!\nCalling rotate_n with {names_interp[o]} interpolation did not produce the correct result. Check the images below:')
        img_list.append(rotated[i])
        img_list.append(reference[i])
        img_list.append(reference[i] - rotated[i])
        title_list.append(names[i])
        title_list.append(names[i] + ' (correct)')
        title_list.append('Difference')
        err_count += 1
if err_count > 0:
    view = viewer(test_img, title=['Test image'], subplots=(1,1))
    view_err = viewer(img_list, title=title_list, subplots=(err_count,3))
else:
    view = viewer([test_img] + rotated, title=['Test image'] + names, subplots=(2,2))
    print('Nice! Your rotate_n function passed the sanity check.')


In the next cell, we play around with the `rotate_n` function to get a feeling for why a good interpolation is so important.
Nearest neighbor interpolation artifacts are particularly pronounced when the angle of rotation is small and the transformation is applied often.
As an example, try setting  $\alpha=1.5^\circ$, $n=20$.

In [ ]:
from ipywidgets import widgets


alpha_slider = widgets.FloatSlider(value=0, min=-90, max=90, step=0.5, description='α(degrees)')
n_slider = widgets.IntSlider(value=5, min=1, max=20, description='n')
order_dropdown = widgets.Dropdown(options=['0: Nearest neighbor', '1: Linear', '3: Cubic'], value='1: Linear', description='order:', disabled=False)
order_dictionary = {'0: Nearest neighbor':0, '1: Linear':1, '3: Cubic':3}
button = widgets.Button(description='Apply rotations')


def rotate_n_callback(img):
    return rotate_n(img, np.deg2rad(alpha_slider.value), n_slider.value, order_dictionary[order_dropdown.value])


plt.close("all")
view = viewer(
    [alsace], title='Alsace',
    new_widgets=[order_dropdown, alpha_slider, n_slider, button], 
    callbacks=[rotate_n_callback], widgets=True
)

In the next cell, we plot the signal to noise ratio of the three interpolators when rotating an image $n$ times by $35^{\circ}$ and then $n$ times by $-35^{\circ}$.

In [ ]:
def snr(img, ref):
    return 10*np.log10(np.sum(img ** 2) / np.sum((ref - img) ** 2))


alpha = np.deg2rad(35)
ns = np.arange(1, 11, dtype=int)
snr_nn  = [snr(rotate_n(rotate_n(alsace, alpha, n, 0), -alpha, n, 0), alsace) for n in ns]
snr_lin = [snr(rotate_n(rotate_n(alsace, alpha, n, 1), -alpha, n, 1), alsace) for n in ns]
snr_cub = [snr(rotate_n(rotate_n(alsace, alpha, n, 3), -alpha, n, 3), alsace) for n in ns]

plt.close('all')
plt.figure()
plt.plot(ns, snr_nn,  label='nn')
plt.plot(ns, snr_lin, label='linear')
plt.plot(ns, snr_cub, label='cubic')
plt.legend()
plt.title('SNR of images rotated n times by $35^{\\circ}$ and back')
plt.grid()
plt.ylabel('SNR [dB]')
plt.xlabel('n')
plt.show()

# Image registration

Now we use image transformations to align two images in terms of rotation and zoom.
We use an example of a map and a satellite image of Sicily, where the aim is to align the satellite image to the map using `transform`.

In [ ]:
sicilia_photo, sicilia_map = [imageio.imread(f'images/{name}.png') for name in ['sicilia_photo', 'sicilia_map']]
fig, axs = plt.subplots(1, 2)
for im, title, ax in zip([sicilia_photo, sicilia_map], ['Satellite image', 'Map'], axs):
    ax.imshow(im, cmap='gray')
    ax.set_title(title)
    ax.axis('off')

plt.show()

## Implementation (2 points)

To accurately match the satellite image to the map, we determine angle, scale, and center of the transformation through two point correspondences.
For this, we identify at least two features present in both images.
In our case, the island of Malta and the city of Messina are good candidates.
Let $\mathbf{u_1}$ and $\mathbf{u_2}$, and $\mathbf{v_1}$ and $\mathbf{v_2}$ denote the locations of the first and second feature in the map and the satellite image, respectively.

For example, the center point can be calculated by re-arranging the transformation equation

$$
(\mathbf{v}-\mathbf{c}) = \mathbf{A}(\mathbf{u} - \mathbf{c}) \mbox{ to obtain }
\mathbf{c} = (\mathbf{A} - \mathbf{I})^{-1}(\mathbf{A}\mathbf{u} - \mathbf{v} )\,,
$$

where $\mathbf{A}$ is the rotation matrix and $\mathbf{u}$, $\mathbf{v}$ can either be $\mathbf{u_1}$, $\mathbf{v_1}$ or $\mathbf{u_2}$, $\mathbf{v_2}$.

**For 3 points**, implement `register`, which performs the image registration given the four points $\mathbf{u_1}$, $\mathbf{u_2}$, $\mathbf{v_1}$ and $\mathbf{v_2}$. The function takes as input parameters

 * `img`: the original image, 
 * `u1`: an array containing the coordinates of the first point in the original image, 
 * `u2`: an array containing the coordinates of second point in the original image,
 * `v1`: an array containing the coordinates of the first point in the target image, 
 * `v2`: an array containing the coordinates of the second point in the target image,
 
and outputs

 * `out`: the registered image.

You will have to compute the `scaling`, `angle`, and `center` of rotation parameters for the `transform` call on the last line.
    
💡 *Hint: You might want to use some of the following functions: [`np.linalg.norm`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html), [`np.arctan2`](https://numpy.org/doc/stable/reference/generated/numpy.arctan2.html), [`np.dot`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html), [`np.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) and [`np.identity`](https://numpy.org/doc/stable/reference/generated/numpy.identity.html). Check their documentation (click on their names here) if you are not familiar with them.*

In [ ]:
def register(img, u1, v1, u2, v2):
    # You will have to recompute these parameters
    angle = 0
    scale = 1
    center = [0, 0]

    o = u2 - u1
    t = v2 - v1

    # YOUR CODE HERE
    
    return transform(img, angle=angle, scale=scale, center=center, order=3)

Run the cell below for a quick sanity check. The two reference points in the test image are the leftmost and rightmost points of the long line.

In [ ]:
target = np.zeros((101, 101)) 
target[50, 10:90] = 1
target[20:50, 50] = 1

original = transform(target, angle=np.deg2rad(60), scale=0.6, center=(30, 40), order=3)

u1 = np.array((50, 42))
u2 = np.array((21, 39))
v1 = np.array((50, 10))
v2 = np.array((20, 50))

registered = register(original, u1, v1, u2, v2)
reg_corr = transform(original, angle=np.deg2rad(-59.0362), scale=1.715, center=(28.3784, 40.2703), order=3)
if np.allclose(registered, reg_corr, atol=1e-3):
    print('Congrats! Your register function seems correct!')
else:
    print('WARNING!\nThe registration function is not yet correct. Look at the images below to see the differences.')
plt.close('all')
view = viewer([original, target, registered, reg_corr],
              title=['Original test image', 'Target test image', 'Your registered image', 'Correctly registered image'], 
              subplots=(2,2))


## Experimentation

In this section, we are going to make direct use of your previous implementations to map the `sicilia_map` and `EPFL_map` images to `sicilia_photo` and `EPFL_photo`, respectively. For this purpose, we provide you with a viewer that allows you to select the mapping points and perform the transformations. Make sure that the reference points you select in the original image correspond to the same reference points in the target image.

    
The points $\mathbf{u_1}$, $\mathbf{u_2}$, $\mathbf{v_1}$ and $\mathbf{v_2}$ are specified using the viewer with `clickable=True`.
Two images will be displayed side by side, and the selected point are assigned in the following order:

1. $u_1$ on the first image,
1. $v_1$ on the second image,
1. $u_2$ on the first image,
1. $v_2$ on the second image

In case want to restart, you can either finish the selection and restart (happens automatically), click on the button *Reset* (losing any zoom and pan you may have done), or re-run the entire cell.

Marks will appear at the clicked point location on the selected image as well as the following display in the statistics panel:
`['click:i,x=, y=']`.
Once the four points are selected, the `Register Image` button undert the `Extra Widgets` performs the registration.

In [ ]:
button = widgets.Button(description='Register Image')

def save_points(img, coords): 
    if len(coords) == 4: 
        u1 = np.array([coords[0]['y'], coords[0]['x']])
        v1 = np.array([coords[1]['y'], coords[1]['x']])
        u2 = np.array([coords[2]['y'], coords[2]['x']])
        v2 = np.array([coords[3]['y'], coords[3]['x']])
        #print(u1, v1, u2, v2)
        return register(img, u1, v1, u2, v2)
    else: 
        print('Select ', 4 - len(coords), 'points more.')
        return img

plt.close('all')
sicily_view = viewer([sicilia_photo, sicilia_map], title=['Sicilia map','Sicilia photo'], new_widgets=[button],
                     callbacks=[save_points], widgets=True, subplots=(1,2), clickable=True)

Now let's try it with the EPFL campus images. This time we align the map to the satellite image.

In [ ]:
epfl_map, epfl_photo = [imageio.imread(f'images/{name}.png') for name in ['EPFL_map', 'EPFL_photo']]
plt.close('all')
epfl_viewer = viewer([epfl_map, epfl_photo], title=['EPFL map', 'EPFL photo'], new_widgets=[button],
                     callbacks=[save_points], widgets=True, subplots=(1,2), clickable=True)

# Image distortion

| Eiffel Tower              | Distorted Eiffel Tower               |
|------------------------|-----------------------|
| <img src="images/distort1.png" width="220px" alt="Fowarding"> | <img src="images/distort2.png" width="220px" alt="Routing"> |


In this section, we implement `distort` that relates the position $\mathbf{x_1}=(i_1, j_1)$ in the target image to the position $\mathbf{x_2}=(i_2, j_2)$ in the original image with the pixel-wise relation

$$
i_2 = i_1 + \delta(i_1,j_1)(u_i - v_i) \mbox{, and }j_2 = j_1 + \delta(i_1,j_1) (u_j - v_j)\,.
$$

Here, two points $\mathbf{u} = (u_i, u_j)$ and $\mathbf{v} = (v_i, v_j)$ specify the direction of the displacement, while the magnitude is regulated by $\delta(i_1,j_1)$ as

$$
\delta(i_1,j_1) = \exp\left(-\frac{(i_1 - v_i)^2 + (j_1 - v_j)^2}{k^2}\right)\,.
$$

Note that in the particular case where $\mathbf{x_1}=\mathbf{v}$ in the target image, the resulting value in the original image is taken from $\mathbf{x_2}=\mathbf{u}$.

## Implementation (3 points)

Make sure to understand the proposed operation before starting to code, and **for 1 point** answer the following MCQ:

* Q1: What happens when we **increase** the parameter $k$?

    1. The direction of the distortion will be more horizontal,
    2. the direction of the distortion will be more vertical,
    3. a larger area of the image will be affected by the distortion, or
    4. a smaller area of the image will be affected by the distortion.

⚠️ **Note: To answer, change the variable `answer` in the cell below to the number corresponding to your choice. Then run the cell below to check that your answer is valid.**

In [ ]:
# Modify the variable answer
answer = None
# YOUR CODE HERE

In [ ]:
# Sanity check
if not answer in [1, 2, 3, 4]:
    print('WARNING!\nValid answers are 1, 2, 3 and 4.')

**For 2 points**, complete the function `distort` in the next cell. The function `distort(i, j, u_i, u_j, v_i, v_j, k)` takes as input parameters

* `i`: $i_1$,
* `j`: $j_1$,
* `u_i`: $u_i$,
* `u_j`: $u_j$,
* `v_i`: $v_i$,
* `v_j`: $v_j$, and
* `k`: $k$,
    
and returns

* `i_new`, `j_new`: $i_2$, $j_2$.

⚠️ **Note: Make sure to calculate `i_new` and `j_new` separately (don't perform any vector calculations). We will call the `distort()` function passing two-dimensional arrays in `i` and `j`, so putting them together in a vector would probably fail and/or complicate the implementation unnecessarily.**

In [ ]:
def distort(i, j, u_i, u_j, v_i, v_j, k):
    i_new = i
    j_new = j
    
    # YOUR CODE HERE
    
    return i_new, j_new

Let us perform a sanity check on the `distort` function.

In [ ]:
i_corr = np.array([
    [ 1.711,  1.85 ,  1.938,  1.969,  1.938],
    [ 2.738,  2.879,  2.969,  3.   ,  2.969],
    [ 3.711,  3.85 ,  3.938,  3.969,  3.938],
    [ 4.632,  4.765,  4.85 ,  4.879,  4.85 ],
    [ 5.51 ,  5.632,  5.711,  5.738,  5.711]
])
j_corr = np.array([
    [-1.711, -0.85 ,  0.062,  1.031,  2.062],
    [-1.738, -0.879,  0.031,  1.   ,  2.031],
    [-1.711, -0.85 ,  0.062,  1.031,  2.062],
    [-1.632, -0.765,  0.15 ,  1.121,  2.15 ],
    [-1.51 , -0.632,  0.289,  1.262,  2.289]
])

u = [3, 1]
v = [1, 3]
k = 8

idxs = np.round(np.fromfunction(lambda i, j: distort(i, j, u[0], u[1], v[0], v[1], k), shape=(5,5)), 3)
print(f'Your distorted i-coordinates:\n{idxs[0]}\n\nYour distorted j-coordinates:\n{idxs[1]}\n')

if not (np.allclose(i_corr, idxs[0], atol=1e-3) and np.allclose(j_corr, idxs[1], atol=1e-3)):
    print('You do not pass the sanity check')


## Experimentation

Now we will again create an interactive viewer so that you can play around with the `distort()` function. Like in the `register()` experimentation, you can click on the image to define the points $\mathbf{u}$ and $\mathbf{v}$ and then click on `Distort` to apply the distortion. Additionally, you can choose the value for the parameter $k$ with the slider. You will need to click on the `Distort` button again to apply any new value for $k$.

If you look at the code below, you can see that we apply your distort function to all pixel locations of the eiffel image, using [`np.fromfunction`](https://numpy.org/doc/stable/reference/generated/numpy.fromfunction.html). This gives us all the distorted pixel indices for the entire image, which we can then interpolate with a cubic spline interpolation by using the function [`ndimage.map_coordinates`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.map_coordinates.html). If you're interested, you can go check out their documentation.

In [ ]:
k_slider = widgets.IntSlider(value=64, min=1, max=256, description='k')
button = widgets.Button(description = 'Distort')
eiffel = imageio.imread('images/eiffel.png') / 255.

def distort_callback(img, coords): 
    if len(coords) == 2: 
        # Get distortion parameters
        k = k_slider.value
        u = (coords[0]['y'], coords[0]['x'])
        v = (coords[1]['y'], coords[1]['x'])
        idxs = np.fromfunction(lambda i, j: distort(i, j, u[0], u[1], v[0], v[1], k=k), shape=img.shape)
        return nd.map_coordinates(img, idxs, order=3, mode='constant').reshape(img.shape)
    else:
        print('Select ', 2 - len(coords), 'points more.')
        return img


plt.close('all')
distort_view = viewer(eiffel, widgets=True, new_widgets=[k_slider, button], callbacks=[distort_callback], clickable=True, line=True)

🎉 Congratulations on finishing the second part of the Geometric Transformation lab!! 🎉

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to Moodle, **in a zip file with the other notebook of this lab.**

* Keep the name of the notebook as: *2_gt_applications.ipynb*,
* Name the zip file: *geometric_transformation_lab.zip*.